In [1]:
import spacy
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
)
import pandas as pd
import numpy as np
import pandas as pd
import re
import spacy
from spacy.tokens import DocBin
from spacy.cli.train import train

In [2]:
nlp = spacy.blank("pt")
df_train = pd.read_csv('../../../dataset/v1/train.csv')
df_test = pd.read_csv('../../../dataset/v1/test.csv')

labels = df_train.columns[3:].to_list()
X_train = list(range(0, df_train.shape[0]))
y_train = df_train[labels].to_numpy()
y_test = df_test[labels].to_numpy()

In [3]:
db = DocBin()
for i in range(df_train.shape[0]):
    doc = nlp(df_train.iat[i, 2])
    for j, label in enumerate(df_train.columns[3:].to_list()):
        doc.cats[label] = df_train.iat[i, 3+j]
    db.add(doc)
db.to_disk('./dataset/train.spacy')

In [4]:
# dummy data for dev
db = DocBin()
doc = nlp('I like this game, it has decent plot and good graphic. Also run on Steam Deck smoothly')
doc.cats['label_recommended'] = 1
doc.cats['label_story'] = 1
doc.cats['label_gameplay'] = 0
doc.cats['label_visual'] = 1
doc.cats['label_audio'] = 0
doc.cats['label_technical'] = 1
doc.cats['label_price'] = 0
doc.cats['label_suggestion'] = 1
db.add(doc)
db.to_disk('./dataset/dev.dummy.spacy')

In [5]:
# ignore score since we use dummy data for dev
train('./config.cfg', output_path='./output')

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TEXTC...  CATS_SCORE  CATS_MACRO_P  CATS_MACRO_R  CATS_MACRO_F  SCORE 
---  ------  ------------  -------------  ----------  ------------  ------------  ------------  ------
  0       0          0.02           0.35        0.00         25.00         25.00         25.00    0.25
  0     200          1.13          52.73        0.00         12.50         12.50         12.50    0.12
  0     400          1.17          48.14        0.00          0.00          0.00          0.00    0.00
  0     600          1.52          42.50        0.00         12.50         12.50         12.50    0.12
  0     800          1.07          40.86        0.00         12.50         12.

In [6]:
nlp = spacy.load('./output/model-last')
nlp.pipe_names

['tok2vec', 'textcat_multilabel']

In [7]:
nlp('I like this game so much').cats

{'label_recommended': 0.19961223006248474,
 'label_story': 0.008869870565831661,
 'label_gameplay': 0.4891369044780731,
 'label_visual': 0.02876739390194416,
 'label_audio': 0.0017880821833387017,
 'label_technical': 0.033372193574905396,
 'label_price': 0.215810626745224,
 'label_suggestion': 0.06886571645736694}

In [8]:
y_pred = np.zeros(shape=y_test.shape)
for i, doc in enumerate(nlp.pipe(df_test['cleaned_review'].to_list())):
    y_pred[i, :] = list(doc.cats.values())
mask_pos = y_pred > 0.5
y_pred[mask_pos] = 1.0
y_pred[~mask_pos] = 0.0
y_pred

array([[1., 0., 1., ..., 1., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 1., 0., 0.],
       [0., 0., 1., ..., 1., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [9]:
def evaluate(y_test, y_pred, labels):
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Overall accuracy: {accuracy}')
    for idx, label in enumerate(labels):
        label_accuracy = accuracy_score(y_test[:, idx], y_pred[:, idx])
        print(f'Accuracy {label}: {label_accuracy}')

    f1 = f1_score(y_test, y_pred, average='macro')
    print(f'F1 macro: {f1}')
    print(
        classification_report(y_test, y_pred, target_names=labels, digits=4, zero_division=0)
    )

evaluate(y_test, y_pred, labels)

Overall accuracy: 0.195
Accuracy label_recommended: 0.805
Accuracy label_story: 0.79
Accuracy label_gameplay: 0.82
Accuracy label_visual: 0.735
Accuracy label_audio: 0.89
Accuracy label_technical: 0.76
Accuracy label_price: 0.765
Accuracy label_suggestion: 0.89
F1 macro: 0.6299451658099359
                   precision    recall  f1-score   support

label_recommended     0.8303    0.9257    0.8754       148
      label_story     0.7527    0.7865    0.7692        89
   label_gameplay     0.8278    0.9675    0.8922       154
     label_visual     0.6441    0.8736    0.7415        87
      label_audio     0.7101    0.9608    0.8167        51
  label_technical     0.5692    0.6491    0.6066        57
      label_price     0.5000    0.2553    0.3380        47
 label_suggestion     0.0000    0.0000    0.0000        21

        micro avg     0.7413    0.8104    0.7743       654
        macro avg     0.6043    0.6773    0.6299       654
     weighted avg     0.7118    0.8104    0.7524       654